<a href="https://colab.research.google.com/github/RouaBenYahia/RAG/blob/main/Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/FineTuning-Rag

/content/drive/MyDrive/FineTuning-Rag


In [ ]:
!ls

app.ipynb  Finetuningp1.ipynb  htmltemplate.py	Streamlit.ipynb
app.py	   Finetuningp2.ipynb  __pycache__	Streamlit.py


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/FineTuning-Rag')


In [ ]:
!pip install -q streamlit langchain PyPDF2 faiss-cpu tiktoken huggingface-hub
!pip install -U langchain_community langchain-huggingface
!pip install python-docx
!sudo apt-get install tesseract-ocr -y
!pip install pytesseract pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 131.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not 

In [ ]:
from google.colab import userdata
api_key=userdata.get('huggingface')
from huggingface_hub import login
login(token=api_key)


In [ ]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦
added 22 packages in 2s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦

In [ ]:
%%writefile Streamlit.py
import streamlit as st
from PyPDF2 import PdfReader
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
#from langchain_community.vectorstores import FAISS
from PIL import Image
import pytesseract
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from htmltemplate import css, bot_template, user_template
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.prompts import PromptTemplate
import pandas as pd
import docx
import io
from openpyxl import load_workbook



def extract_text_from_files(uploaded_files):
    all_text = ""

    for uploaded_file in uploaded_files:
        file_name = uploaded_file.name.lower()

        # PDF
        if file_name.endswith(".pdf"):
            try:
                pdf_reader = PdfReader(uploaded_file)
                for page in pdf_reader.pages:
                    text = page.extract_text()
                    if text:
                        all_text += text + "\n"
            except:
                st.warning(f"❌ Failed to read PDF: {uploaded_file.name}")

        # TXT
        elif file_name.endswith(".txt"):
            stringio = io.StringIO(uploaded_file.getvalue().decode("utf-8"))
            all_text += stringio.read() + "\n"

        # CSV
        elif file_name.endswith(".csv"):
            try:
                df = pd.read_csv(uploaded_file)
                all_text += df.to_string(index=False) + "\n"
            except:
                st.warning(f"❌ Failed to read CSV: {uploaded_file.name}")

        # DOCX
        elif file_name.endswith(".docx"):
            try:
                doc = docx.Document(uploaded_file)
                for para in doc.paragraphs:
                    all_text += para.text + "\n"
            except:
                st.warning(f"❌ Failed to read DOCX: {uploaded_file.name}")

        # XLSX
        elif file_name.endswith(".xlsx"):
            try:
                wb = load_workbook(uploaded_file, data_only=True)
                for sheet in wb.sheetnames:
                    ws = wb[sheet]
                    for row in ws.iter_rows(values_only=True):
                        row_text = " | ".join([str(cell) if cell is not None else "" for cell in row])
                        all_text += row_text + "\n"
            except:
                st.warning(f"❌ Failed to read XLSX: {uploaded_file.name}")

        # PNG, JPG (image OCR)
        elif file_name.endswith((".png", ".jpg", ".jpeg")):
            try:
                image = Image.open(uploaded_file)
                text = pytesseract.image_to_string(image, lang='eng')  # ou 'fra' si c’est en français
                all_text += text + "\n"
            except Exception as e:
                st.warning(f"❌ Failed to extract text from image {uploaded_file.name}: {e}")

        else:
            st.warning(f"⚠️ Unsupported file type: {uploaded_file.name}")

    return all_text







def get_text_chuncks(text):
  text_splitter=CharacterTextSplitter(separator="\n",chunk_size=1000,chunk_overlap=300,length_function=len)
  chunks=text_splitter.split_text(text)
  return chunks



def get_vectorstore(text_chunks):
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu"}
)

    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore


def get_conversation_chain(vectorstore):
    #tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
    #model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
    tokenizer = AutoTokenizer.from_pretrained("rouabenyahia/FineTuningModel")
    model = AutoModelForCausalLM.from_pretrained("rouabenyahia/FineTuningModel")

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.0,     # Réponses déterministes
        do_sample=False
    )
    llm = HuggingFacePipeline(pipeline=pipe)

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True
    )

    prompt_template = """
    Tu es un assistant expert. Utilise le contexte ci-dessous pour répondre à la question de l'utilisateur.
    Si la réponse ne se trouve pas dans les documents, dis simplement que tu ne sais pas.

    Contexte:
    {context}

    Question:
    {question}

    Réponse:
    """

    prompt = PromptTemplate.from_template(prompt_template)

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever = vectorstore.as_retriever(search_kwargs={"k": 5}),
        #retriever=vectorstore.as_retriever(),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": prompt}
    )
    return conversation_chain




def handle_userinput(user_question):
    retriever = st.session_state.conversation.retriever
    docs = retriever.get_relevant_documents(user_question)
    #st.write("🔍 Chunks retrouvés par le retriever:", len(docs))

    if len(docs) == 0:
        st.warning("⚠️ Aucun passage du document n’a été trouvé pour cette question. Essayez de reformuler.")

    response = st.session_state.conversation({'question': user_question})
    st.session_state.chat_history = response['chat_history']

    for i, message in enumerate(st.session_state.chat_history):
        content = message.content
        if i % 2 == 1:
            if "Réponse:" in content:
                content = content.split("Réponse:")[-1].strip()
            st.write(bot_template.replace("{{MSG}}", content), unsafe_allow_html=True)
        else:
            st.write(user_template.replace("{{MSG}}", content), unsafe_allow_html=True)







st.set_page_config(
    page_title="Chat with your data",
    page_icon=":books:",
)

st.write(css,unsafe_allow_html=True)

if "conversation" not in st.session_state:
  st.session_state.conversation=None


if "chat_history" not in st.session_state:
  st.session_state.chat_history=None

st.header("Chat with your data :books:")

#user_question=st.text_input("Ask a question about your documents")
#if user_question:
  #handle_userinput(user_question)

user_question = st.text_input("Ask a question about your documents")

if user_question and st.session_state.conversation is not None:
    handle_userinput(user_question)
elif user_question:
    st.warning("📄 Veuillez d'abord uploader vos PDF et cliquer sur 'Process'.")




with st.sidebar:
  st.subheader("Your documents")
  uploaded_files = st.file_uploader(
    "Upload your files (.pdf, .txt, .csv, .docx, .xlsx, .png, .jpg) and click on 'Process'",
    type=["pdf", "txt", "csv", "docx", "xlsx", "png", "jpg", "jpeg"],
    accept_multiple_files=True
)




  if st.button("Process"):
    with st.spinner("Processing"):

      raw_text = extract_text_from_files(uploaded_files)
      if not raw_text.strip():
        st.error("⚠️ Aucun texte n’a été extrait. Les fichiers sont peut-être vides, scannés ou illisibles.")
        st.stop()
      else:
        st.write(f"📄 Nombre de caractères extraits: {len(raw_text)}")



      #st.write(raw_text)
      text_chunks=get_text_chuncks(raw_text)
      #st.write("✅ Nombre de chunks extraits:", len(text_chunks))

      #st.write(text_chunks)
      vectorstore = get_vectorstore(text_chunks)

      #st.write(vectorstore)
      #el conversation chain tawa
      # on peut faire ça
      st.session_state.conversation=get_conversation_chain(vectorstore)
    st.success("✅ The processing of the files is over, you can ask your questions now!")
    #st.session_state.conversation


    #get the pdf text
    #get the text chuncks
    #create the vector store with the embeddings
  #juste el api key mtaa hugging face bech nhothaa


Overwriting Streamlit.py


In [ ]:
!ls

app.ipynb  Finetuningp1.ipynb  htmltemplate.py	Streamlit.ipynb
app.py	   Finetuningp2.ipynb  __pycache__	Streamlit.py


In [ ]:

! wget -q -O - ipv4.icanhazip.com



35.237.252.3


In [ ]:


! streamlit run Streamlit.py & npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇

⠏⠋⠙⠹
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.236.152.101:8501

your url is: https://few-drinks-trade.loca.lt
2025-07-23 18:19:04.069611: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753294744.085452    5333 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753294744.090036    5333 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-23 18:19:04.105792: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the f